## Co-axial drone dynamics
<img src="Drone1.png" width=300>

In this exercise, you will populate the ```CoaxialCopter``` class with three methods. These functions will calculate vertical acceleration $\ddot{z}$, angular acceleration $\ddot{\psi}$ along the $z$ axis and the angular velocities $\omega_1$ and $\omega_2$ of the propellers required to achieve any desired  $\ddot{z}$ and $\ddot{\psi}$. 

We assume that drone only can travel and rotate along the vertical $z$ axis. 

Remember, the positive $z$ axis points downwards. 

In [1]:
%matplotlib inline 
%config InlineBackend.figure_format = 'retina'

import numpy as np 
import math
import matplotlib.pyplot as plt
import matplotlib.pylab as pylab
import jdc
from ExerciseAnswers import Answers

pylab.rcParams['figure.figsize'] = 10, 10

#### Helpful Equations

$$F_z = m \ddot{z}$$

$$M_z = I_z \ddot{\psi}$$

$$ \ddot{z} = g- \frac{k_f}{m}\left( \omega_1^2 +\omega_2^2 \right)   \\
   \ddot{\psi} =\frac{k_m}{I_z} (-\omega_1^2 + \omega_2^2 )  $$

In [2]:
class CoaxialCopter:
    
    def __init__(self, 
                 k_f = 0.1, # value of the thrust coefficient
                 k_m = 0.1, # value of the angular torque coefficient
                 m = 0.5,   # mass of the vehicle 
                 i_z = 0.2, # moment of inertia around the z-axis
                ):
        
        self.k_f = k_f
        self.k_m = k_m
        self.m = m
        self.i_z = i_z
        
        self.omega_1 = 0.0
        self.omega_2 = 0.0
        self.g = 9.81
    
    @property
    def z_dot_dot(self):
        """Calculates current vertical acceleration."""
        
        # TODO: 
        # 1. Calculate the lift force generated by the first 
        #    and second propellers 
        # 2. Calculate the total vertical force acting on the drone 
        # 3. Calculate the vertical acceleration due to the 
        #    total force acting on the drone keep in mind that the 
        #    z-axis is directed downward 

        f_1 = self.k_f * self.omega_1**2
        f_2 = self.k_f * self.omega_2**2
        f_g = self.m * self.g
        f_total = -f_1 - f_2 + f_g

        acceleration = f_total / self.m 

        return acceleration
    
    @property
    def psi_dot_dot(self): 
        """Calculates current rotational acceleration."""
        # TODO: 
        # 1. Calculate the torques generated by both propellers 
        # 2. Calculate the angular acceleration 

        cw_torque = self.k_m * self.omega_1 **2
        ccw_torque = self.k_m * self.omega_2 **2

        net_torque = cw_torque - ccw_torque
        angular_acc = net_torque / self.i_z

        return angular_acc
    
    def set_rotors_angular_velocities(self, linear_acc, angular_acc): 
        """
        Sets the turn rates for the rotors so that the drone
        achieves the desired linear_acc and angular_acc.
        """
        
        # TODO 
        # 1. Calculate the correct values of omega_1 and omega_2
        # 2. Set self.omega_1 and self.omega_2 to those values
        # 3. Don't forget to return omega_1, omega_2
        
        term_1 = self.m * (-linear_acc + self.g) /(2 * self.k_f)
        term_2 = self.i_z * angular_acc/(2 * self.k_m)

        omega_1 = math.sqrt(term_1 - term_2)
        omega_2 = math.sqrt(term_1 + term_2)

        self.omega_1 = -omega_1
        self.omega_2 = omega_2

        return self.omega_1, self.omega_2

In [3]:
# TEST CODE 1

bi = CoaxialCopter()
stable_omega_1,stable_omega_2 = bi.set_rotors_angular_velocities(0.0, 0.0)

print('Drone achieves stable hover with angular velocity of %5.2f' % stable_omega_1, 
      'for the first propeller and %5.2f' % stable_omega_2, 
      'for the second propeller.')

Answers.angular_velocities(bi.m, bi.g, bi.i_z, bi.k_f, bi.k_m, 0.0, 0.0, stable_omega_1, stable_omega_2)

Drone achieves stable hover with angular velocity of -4.95 for the first propeller and  4.95 for the second propeller.


'You calculated correct values'

In [4]:
# TEST CODE 2 - Checking the linear acceleration value

bi.omega_1 = stable_omega_1 * math.sqrt(1.1)
bi.omega_2 = stable_omega_2 * math.sqrt(1.1)

vertical_acceleration = bi.z_dot_dot
print('Increase by %5.2f' % math.sqrt(1.1),
      'of the propeller angular velocity will result in',
      '%5.2f' % vertical_acceleration, 
      'm/(s*s) vertical acceleration.' )

Answers.linear_acceleration(bi.m, bi.g, bi.k_f, bi.omega_1, bi.omega_2, vertical_acceleration)

Increase by  1.05 of the propeller angular velocity will result in -0.98 m/(s*s) vertical acceleration.


'You calculated the acceleration value correctly'

In [5]:
# TEST CODE 3 - checking the angular acceleration
bi.omega_1 = stable_omega_1 * math.sqrt(1.1)
bi.omega_2 = stable_omega_2 * math.sqrt(0.9)

ang_acceleration = bi.psi_dot_dot
print('Increase in %5.2f'%math.sqrt(1.1),' of the angular velocity for the first propeller and',
      ' decrease of the angular velocity of the second propeller by %f.2f'%math.sqrt(0.9),' will result in',
      '%5.2f'%ang_acceleration, 'rad/(s*s) angular acceleration.' )

Answers.angular_acceleration(bi.i_z, bi.k_m, bi.omega_1, bi.omega_2, ang_acceleration)

Increase in  1.05  of the angular velocity for the first propellr and  decrease of the angular velocity of the second propellr by 0.948683.2f  will result in  2.45 rad/(s*s) angular acceleration.


'You calculated the angular acceleration correctly'